In [51]:
import os
os.environ["OPENAI_API_KEY"] = "sk-proj-2oCiQGD-PYRYC7Kz0Wo39d1V8ZaZg6N0yiXPGvUwTrVT7w3XqpmMRf0MMmTvGl_w_Ztb41FXYNPgA"

# Real Time Currency Conversion (between any two currencies)

In [52]:
!pip install -q langchain-openai langchain-core requests

In [53]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [100]:
#Tool create
from langchain_core.tools import InjectedToolArg
from typing import Annotated
@tool

def get_conversion_factor(base_currency:str,target_currency:str) ->float:
  """This function  fetches the currency conversion fator between a given base currency and a target currency
  """
  url= f"https://v6.exchangerate-api.com/v6/e8041040d1f0e/pair/{base_currency}/{target_currency}"
  response = requests.get(url)
  return response.json()
@tool
def convert(base_currency: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:

  """given a currency conversion rate this function calculates the target currency value from a given base currency value
  """
  return base_currency*conversion_rate

In [101]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1749081601,
 'time_last_update_utc': 'Thu, 05 Jun 2025 00:00:01 +0000',
 'time_next_update_unix': 1749168001,
 'time_next_update_utc': 'Fri, 06 Jun 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 85.9019}

In [102]:
convert.invoke({'base_currency':10,'conversion_rate':85.9019})

859.019

In [103]:
#tool binding
llm = ChatOpenAI()

In [104]:
llm_with_tools = llm.bind_tools([get_conversion_factor,convert])   #binded two tools

In [105]:
messages = [HumanMessage('What is the conversion factor between USD and INR, and based on that can you convert 10 USD to INR')]

In [106]:
messages

[HumanMessage(content='What is the conversion factor between USD and INR, and based on that can you convert 10 USD to INR', additional_kwargs={}, response_metadata={})]

In [107]:
ai_messages = llm_with_tools.invoke(messages)

In [108]:
messages.append(ai_messages)

In [109]:
ai_messages.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'INR'},
  'id': 'call_VhvrS8KljC8kHFJz1Ga2kGNX',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency': 10},
  'id': 'call_6yvuy0MMGu421TtgfsXe9na0',
  'type': 'tool_call'}]

In [110]:
import json

for tool_call in ai_messages.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)



In [96]:
messages

[HumanMessage(content='What is the conversion factor between USD and INR, and based on that can you convert 10 USD to INR', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_kbNY7GkVHdh3izgOZxmHBc6k', 'function': {'arguments': '{"base_currency": "USD", "target_currency": "INR"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'call_bigRJRZUjuNo21Io7C3aqmuB', 'function': {'arguments': '{"base_currency": 10}', 'name': 'convert'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 123, 'total_tokens': 174, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BewydmjD8ikMWdeGGc5dNduArO2fY', 'service_tier': 'default', 

In [111]:
llm_with_tools.invoke(messages).content

'The conversion factor between USD and INR is 85.9019. \n\nTherefore, 10 USD is equivalent to 859.019 INR.'